# Calcluate Intertial Matrix for a rocket

In [1]:
import sympy as sp
import sympy.physics.mechanics as me 
import numpy as np
from sympy.utilities.iterables import flatten

sp.init_printing(use_latex='mathjax')

In [2]:
Ixx, Ixy, Ixz = sp.symbols('I_xx, I_xy, I_xz')
Iyx, Iyy, Iyz = sp.symbols('I_yx, I_yy, I_yz')
Izx, Izy, Izz = sp.symbols('I_zx, I_zy, I_zz')


In [3]:
I = sp.Matrix([[Ixx, -Ixy, -Ixz],[-Iyx, Iyy, -Iyz],[-Izx, -Izy, Izz]])

In [4]:
I

⎡ Iₓₓ   -I_xy  -I_xz⎤
⎢                   ⎥
⎢-I_yx  I_yy   -I_yz⎥
⎢                   ⎥
⎣-I_zx  -I_zy  I_zz ⎦

# Point mass $m$ at $(x,y,z)$

See https://physics.stackexchange.com/questions/614094/inertia-tensor-formula-for-point-masses-in-rigid-assembly for an explanation


We are looking for the inertial matrix $I$ :

$$
I = \left[\begin{matrix}m \left(y^{2} + z^{2}\right) & - m x y & - m x z\\- m x y & m \left(x^{2} + z^{2}\right) & - m y z\\- m x z & - m y z & m \left(x^{2} + y^{2}\right)\end{matrix}\right]
$$

where $m$ is the point mass and $r$ is the vector from the center of rotation (usuall CG) to the mass:

$$
r = \left[\begin{matrix}x\\y\\z\end{matrix}\right]
$$

In [5]:
# this is the inertial coordinate system
N = me.ReferenceFrame('N')

In [6]:
# define the symbols
x, y, z, m = sp.symbols('x, y, z, m')

In [7]:
r = x*N.x + y*N.y + z*N.z

In [8]:
r

x n_x + y n_y + z n_z

## Formula to calculate I at $(x,y,z)$

Point mass (m) located at $(x,y,z)$ in the inertial coordinate system (N)

$$I = m(\vec{r} \cdot \vec{r} - \vec{r} \otimes \vec{r})$$


In [9]:
# see the reference above for the origin of this derivation
Ipoint_mass_m = m*(sp.eye(3)*(r.to_matrix(N).T * r.to_matrix(N))[0] - r.to_matrix(N)*r.to_matrix(N).T)  

In [10]:
Ipoint_mass_m

⎡  ⎛ 2    2⎞                          ⎤
⎢m⋅⎝y  + z ⎠    -m⋅x⋅y       -m⋅x⋅z   ⎥
⎢                                     ⎥
⎢               ⎛ 2    2⎞             ⎥
⎢  -m⋅x⋅y     m⋅⎝x  + z ⎠    -m⋅y⋅z   ⎥
⎢                                     ⎥
⎢                            ⎛ 2    2⎞⎥
⎣  -m⋅x⋅z       -m⋅y⋅z     m⋅⎝x  + y ⎠⎦

### Create a python function from this formula

With the follwing function signature:

`func(m, x, y, z)`

Where:
- $m$ : point mass
- $x,y,z$ : the coordinates of the point in the inertial coordinate system

In [11]:

Ipoint_mass_eval = sp.lambdify((m, x, y, z), Ipoint_mass_m, 'numpy')

In [12]:
help(Ipoint_mass_eval)

Help on function _lambdifygenerated:

_lambdifygenerated(m, x, y, z)
    Created with lambdify. Signature:
    
    func(m, x, y, z)
    
    Expression:
    
    Matrix([[m*(y**2 + z**2), -m*x*y, -m*x*z], [-m*x*y, m*(x**2 + z**2),...
    
    Source code:
    
    def _lambdifygenerated(m, x, y, z):
        return array([[m*(y**2 + z**2), -m*x*y, -m*x*z], [-m*x*y, m*(x**2 + z**2), -m*y*z], [-m*x*z, -m*y*z, m*(x**2 + y**2)]])
    
    
    Imported modules:



In [13]:
# test the function: 
# this finds I of two 1kg masses located respectively at x=1 and x=-1
Ipoint_mass_eval(1, 1, 0, 0) + Ipoint_mass_eval(1, -1, 0, 0)

array([[0, 0, 0],
       [0, 2, 0],
       [0, 0, 2]])

# Class describing a body
It will be useful to create a simple class to describe bodies that make up a part

A body is described by in a local (body) coordinate system by:
1. a mass ($m$)
1. location of the CG as a vector ($r$) in the body coordinate system $r = (x,y,z)$
1. a moment of inertia matrix about the CG ($I_{CG}$)

In [14]:
class Body():
  """A simple class to describe a single body. A body is modeled a point mass m located at r with 
  an optional rotational inertia matrix (inertia_matrix). In the case of a point mass the inertia 
  matrix is assumed to be a 3x3 matrix of zeros
  Params:
  =======
  m (np.float for sympy.core.symbol.Symbol) - the mass 
  r (sympy.Matrix) - the vector to the point mass in the body reference frame (body coordinates)
  inertia_matrix (numpy.array or sympy.Matrix) - the inertia matrix of the body arounds its center of gravity.
  
  Returns:
  ========
  body (Body)
  """
  
  
  def __init__(self, mass=None, r=sp.Matrix([0,0,0]), inertia_matrix=sp.Matrix(np.zeros([3,3]))):
    
    self.m = mass
    if type(r) == type(sp.Matrix()):
      self.r = r
    else:
      self.r = sp.Matrix(r)
  
    if type(inertia_matrix) == type(sp.Matrix()):
      self.im = inertia_matrix
    else: 
      self.im = sp.Matrix(inertia_matrix)
                              
    

## Mass 1
point mass of 1$kg$ at $ r=[1, 0, 0]$

i.e. the inertial matrix for a point mass is zeros

In [15]:
m1 = Body(1,  sp.Matrix([1,0,0]))

## Mass 2
point mass of 1$kg$ at $ r=[-1, 0, 0]$


In [16]:
m2 = Body(1,  sp.Matrix([-1,0,0]))

# Calc the CG of a list of bodies

In [17]:
def calcCG(pm_list):
  """Function to calculate the CG of a list of bodies.
  Param:
  ======
  pm_list (list(Body)) - list of Body objects
  Returns:
  ========
  CG - vector to the CofG"""
  
  # this is a hack to create a variable of type PointMass.r
  m_times_r = pm_list[0].r * 0
  m_tot = 0
  
  for pm in pm_list:
    m_tot += pm.m
    m_times_r += pm.m*pm.r
    
  return m_times_r/m_tot
    

In [18]:
# for m1 and m2 as specified above this should return zero
calcCG([m1,m2])

⎡0⎤
⎢ ⎥
⎢0⎥
⎢ ⎥
⎣0⎦

In [19]:
# cg should be [1/3, 0, 0].T
m3 = Body(2,  sp.Matrix([1,0,0]))
m4 = Body(1,  sp.Matrix([-1,0,0]))
cg = calcCG([m3, m4])
cg

⎡1/3⎤
⎢   ⎥
⎢ 0 ⎥
⎢   ⎥
⎣ 0 ⎦

# Calc I in global coords
See here for a reference http://www.kwon3d.com/theory/moi/triten.html   
Check out equations 13 and 14

$I' = I_{CG} + I_{translation}$


In [20]:
def calcI(body_list, cg=None):
  """Function to calculate the rotational inertial matrix for a list of point masses.
  Param:
  pm_list (list(PointMass)) - list of PointMass objects
  cg (sympy.Matrix) - the position of the Center of Gravity in world coordinates
  Returns:
  I (numpy.array) - the rotational inertial matrix"""
  
  # check parameters:
  if not type(body_list)==list:
    print('Requires a list of bodies as input parameter')
    return
  elif not type(body_list[0])==Body:
    print('First element in list is not of type "Body"')
    return
  
  # initialize the result array
  I = np.zeros([3,3])
  
  # if cg == None then calulate the cg first
  
  if cg==None or cg=='calculate':
    print('Calculating CG for body list')
    cg = calcCG(body_list)
  
  # the inertial matrix is the sum of all the individual body inertial matrices
  for pm in body_list:
    # First add the inertia matrix around the CG in body coords
    I = np.add(I,pm.im)
    
    # Then add the inertia matrix of the point body in global coords
    I = np.add(I, Ipoint_mass_eval(pm.m, *flatten(pm.r-cg)))
    
    
  return I
  

# Test
Two masses of 1kg each at [0,0,1] should be equivalent to one mass of 2kg

In [21]:
m1 = Body(2,  sp.Matrix([0,0,1]))
m2 = Body(1,  sp.Matrix([0,0,1]))
m3 = Body(2,  sp.Matrix([0,0,-1]))
m4 = Body(1,  sp.Matrix([0,0,-1]))
m5 = Body(3,  sp.Matrix([0,0,1]))
m6 = Body(3,  sp.Matrix([0,0,-1]))


In [22]:
cg = calcCG([m1, m2, m3, m4])

In [23]:
cg

⎡0⎤
⎢ ⎥
⎢0⎥
⎢ ⎥
⎣0⎦

In [24]:
calcI([m1, m2, m3, m4], cg='calculate')

Calculating CG for body list


array([[6, 0, 0],
       [0, 6, 0],
       [0, 0, 0]], dtype=object)

In [25]:
calcI([m5, m6], cg='calculate')

Calculating CG for body list


array([[6, 0, 0],
       [0, 6, 0],
       [0, 0, 0]], dtype=object)

In [26]:
calcI([m1, m2, m3], cg='calculate')

Calculating CG for body list


array([[24/5, 0, 0],
       [0, 24/5, 0],
       [0, 0, 0]], dtype=object)

In [27]:
# try the same in diffent planes

In [28]:
m1 = Body(1,  sp.Matrix([1,0,1]))
m2 = Body(1,  sp.Matrix([1,0,1]))
m3 = Body(1,  sp.Matrix([-1,0,-1]))
m4 = Body(1,  sp.Matrix([-1,0,-1]))
m5 = Body(2,  sp.Matrix([1,0,1]))
m6 = Body(2,  sp.Matrix([-1,0,-1]))


In [29]:
calcCG([m1, m2, m3, m4])

⎡0⎤
⎢ ⎥
⎢0⎥
⎢ ⎥
⎣0⎦

In [30]:
calcI([m1, m2, m3, m4], cg='calculate')

Calculating CG for body list


array([[4, 0, -4],
       [0, 8, 0],
       [-4, 0, 4]], dtype=object)

In [31]:
calcI([m5, m6], cg='calculate')

Calculating CG for body list


array([[4, 0, -4],
       [0, 8, 0],
       [-4, 0, 4]], dtype=object)

# Moment of Inertia for selected primitives

## Inertia of a thick walled Cylinder

The most generalized primitive is a thick walled cylinder as it can describe:
- solid cylinder
- solid disc
- ring
- thin walled cylinder

<img src="1920px-Moment_of_inertia_thick_cylinder_h.svg.png" alt="Drawing" style="width: 200px;"/>

https://en.wikipedia.org/wiki/List_of_moments_of_inertia

In [32]:
l, r1, r2, m = sp.symbols('l, r1, r2, m')

In [33]:
I_cyl = sp.Matrix([[m*(3*(r2**2+r1**2)+4*l**2)/12, 0, 0],
                   [0, m*(3*(r2**2+r1**2)+4*l**2)/12, 0],
                   [0, 0, m*(r2**2+r1**2)/2]])

In [34]:
# Inertia matrix about the CG of a generalized cylinder
I_cyl

⎡  ⎛   2       2       2⎞                                         ⎤
⎢m⋅⎝4⋅l  + 3⋅r₁  + 3⋅r₂ ⎠                                         ⎥
⎢────────────────────────             0                    0      ⎥
⎢           12                                                    ⎥
⎢                                                                 ⎥
⎢                            ⎛   2       2       2⎞               ⎥
⎢                          m⋅⎝4⋅l  + 3⋅r₁  + 3⋅r₂ ⎠               ⎥
⎢           0              ────────────────────────        0      ⎥
⎢                                     12                          ⎥
⎢                                                                 ⎥
⎢                                                      ⎛  2     2⎞⎥
⎢                                                    m⋅⎝r₁  + r₂ ⎠⎥
⎢           0                         0              ─────────────⎥
⎣                                                          2      ⎦

### Create a python function from this formula

With the follwing function signature:

` func(m, l, r1, r2)`

Where:
- $m$ : point mass
- $l$ : length (or height) of the cylinder
- $r_1$ : inner radius of the cylinder wall
- $r_2$ : outer radius of the cylinder wall


In [35]:
Icyl_eval = sp.lambdify((m, l, r1, r2), I_cyl)

In [36]:
# test with symbolic variables
cylinder_m = m
cylinder_length = l
cylinder_r2 = r2
cylinder_r1 = r1

# the rotational inertia matrix around the cg of the body
cyl_i = Icyl_eval(cylinder_m, cylinder_length, cylinder_r1, cylinder_r2)
cyl_i

array([[0.0833333333333333*m*(4*l**2 + 3*r1**2 + 3*r2**2), 0, 0],
       [0, 0.0833333333333333*m*(4*l**2 + 3*r1**2 + 3*r2**2), 0],
       [0, 0, 0.5*m*(r1**2 + r2**2)]], dtype=object)

In [37]:
# test with values

cylinder_m = 0.1
cylinder_length = 0.8
cylinder_r2 = 0.025
cylinder_r1 = 0.024

# the rotational inertia matrix around the cg of the cylinder
Icyl_cg = Icyl_eval(cylinder_m, cylinder_length, cylinder_r1, cylinder_r2, )
Icyl_cg

array([[2.13633583e-02, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 2.13633583e-02, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 6.00500000e-05]])

### Translational effect of cylinder CG at z=1

In [38]:
Icyl_t = Ipoint_mass_eval(cylinder_m,0,0,1)
Icyl_t

array([[ 0.1, -0. , -0. ],
       [-0. ,  0.1, -0. ],
       [-0. , -0. ,  0. ]])

In [39]:
# the final inertial matrix of the cylider around its CG and the translation of the CG
Icyl_r = Icyl_cg + Icyl_t
Icyl_r

array([[1.21363358e-01, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 1.21363358e-01, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 6.00500000e-05]])

## Inertia for Sphere

<img src="1920px-Moment_of_inertia_solid_sphere.svg.png" alt="Drawing" style="width: 200px;"/>

https://en.wikipedia.org/wiki/List_of_moments_of_inertia

In [40]:
r, m = sp.symbols('r, m')

In [41]:
I_sphere = 2*m*r**2/5* sp.Matrix([[1, 0, 0],
                   [0, 1, 0],
                   [0, 0, 1]])

In [42]:
# Inertia matrix about the CG of a sphere
I_sphere

⎡     2                ⎤
⎢2⋅m⋅r                 ⎥
⎢──────    0       0   ⎥
⎢  5                   ⎥
⎢                      ⎥
⎢             2        ⎥
⎢        2⋅m⋅r         ⎥
⎢  0     ──────    0   ⎥
⎢          5           ⎥
⎢                      ⎥
⎢                     2⎥
⎢                2⋅m⋅r ⎥
⎢  0       0     ──────⎥
⎣                  5   ⎦

### Create a python function from this formula

With the follwing function signature:

` func(r, m)`

Where:
- $r$ : radius
- $m$ : mass of the sphere


In [43]:
Isphere_eval = sp.lambdify((r, m), I_sphere)

In [44]:
# test with values

sphere_m = 1
sphere_r = 1

# the rotational inertia matrix around the cg of the sphere
Isphere_cg = Isphere_eval(sphere_r, sphere_r)
Isphere_cg

array([[0.4, 0. , 0. ],
       [0. , 0.4, 0. ],
       [0. , 0. , 0.4]])

In [45]:
# Translational effect of cylinder CG at z=1

In [46]:
Isphere_t = Ipoint_mass_eval(sphere_m,0,0,1)
Isphere_t

array([[1, 0, 0],
       [0, 1, 0],
       [0, 0, 0]])

In [47]:
# the final inertial matrix of the cylider around its CG and the translation of the CG
Icyl_r = Isphere_cg + Isphere_t
Icyl_r

array([[1.4, 0. , 0. ],
       [0. , 1.4, 0. ],
       [0. , 0. , 0.4]])

## Inertia for Cube


<img src="Moment_of_inertia_solid_rectangular_prism.png" alt="Drawing" style="width: 200px;"/>

https://en.wikipedia.org/wiki/List_of_moments_of_inertia

In [48]:
h, d, w, m = sp.symbols('h, d, w, m')

In [49]:
I_cube = (m/12) * sp.Matrix([[(h**2+d**2), 0, 0],
                   [0, h**2+w**2, 0],
                   [0, 0, w**2+d**2]])

In [50]:
# Inertia matrix about the CG of a sphere
I_cube

⎡  ⎛ 2    2⎞                          ⎤
⎢m⋅⎝d  + h ⎠                          ⎥
⎢───────────       0            0     ⎥
⎢     12                              ⎥
⎢                                     ⎥
⎢               ⎛ 2    2⎞             ⎥
⎢             m⋅⎝h  + w ⎠             ⎥
⎢     0       ───────────       0     ⎥
⎢                  12                 ⎥
⎢                                     ⎥
⎢                            ⎛ 2    2⎞⎥
⎢                          m⋅⎝d  + w ⎠⎥
⎢     0            0       ───────────⎥
⎣                               12    ⎦

### Create a python function from this formula

With the follwing function signature:

` func(m, w, d, h)`

Where:
- $m$ : mass of the cube
- $w$ : width of the cube (local x-direction)
- $d$ : depth of the cube (local y-direction)
- $h$ : height of the cube (local z-direction)


In [51]:
Icube_eval = sp.lambdify((m, w, d, h), I_cube)

In [52]:
# test with values

cube_w = 1
cube_d = 1
cube_h = 1
cube_m = 1

# the rotational inertia matrix around the cg of the sphere
Icube_cg = Icube_eval(cube_m, cube_w, cube_d, cube_h , )
Icube_cg

array([[0.16666667, 0.        , 0.        ],
       [0.        , 0.16666667, 0.        ],
       [0.        , 0.        , 0.16666667]])

In [53]:
# Translational effect of cylinder CG at z=1

In [54]:
# the final inertial matrix of the cylider around its CG and the translation of the CG
Icube_r = Icube_cg + Ipoint_mass_eval(cube_m,0,0,1)
Icube_r


array([[1.16666667, 0.        , 0.        ],
       [0.        , 1.16666667, 0.        ],
       [0.        , 0.        , 0.16666667]])

In [55]:
# Battery 150g at z=1m

batt_i = Icube_eval(0.15, 0.027, 0.0385, 0.077)

b_batt = Body(0.15,  sp.Matrix([0,0,1]), inertia_matrix=batt_i)

In [56]:
calcCG([b_batt])

⎡ 0 ⎤
⎢   ⎥
⎢ 0 ⎥
⎢   ⎥
⎣1.0⎦

In [57]:
# calc I around its CG 
calcI([b_batt] , cg='calculate')

Calculating CG for body list


array([[9.26406250000000e-5, 0, 0],
       [0, 8.32250000000000e-5, 0],
       [0, 0, 2.76406250000000e-5]], dtype=object)

In [58]:
# calc I around point (0,0,0)
calcI([b_batt], cg=sp.Matrix([0,0,0]))


array([[0.150092640625000, 0, 0],
       [0, 0.150083225000000, 0],
       [0, 0, 2.76406250000000e-5]], dtype=object)

# Real Rocket

Body cooridnate system with [0,0,0] on the ground at the base of the rocket
Assume:
- the rocket stands 1m tall
- the battery (150g) is located at the very top of the rocket $r=(0,0,1)$
- two motors (25g each) are located at z=0.025 $r=(0,0,0.025)$
- flight controller and ESCs (100g) located at $r=(0,0,0.6)$
- rocket body is a 800mm cylinder standing 200mm off the ground with mass 100g diameter 50mm and wall thickness of 1mm  with CG located at $r=(0,0,0.6)$


## Battery

Battery specs: (from [here](https://www.getfpv.com/tattu-r-line-version-3-0-1300mah-4s-120c-lipo-battery.html?utm_source=google&utm_medium=cpc&utm_campaign=DM+-+B+-+PMax+-+Shop+-+SM+-+ALL&utm_content=pmax_x&utm_keyword=&utm_matchtype=&campaign_id=17881616054&network=x&device=c&gc_id=17881616054&gclid=CjwKCAjwgsqoBhBNEiwAwe5w08GJV8UB4zjtOnUhkqqcch9E3ep8q1jL9gUyLSYzRwADdakvn6F9ihoC5TwQAvD_BwE))   

Net Weight(±20g): 150g
Dimensions:77mm x 38.5mm x 27mm (L x W x H)

In [59]:
# Battery 150g at z=1m

b_batt = Body(0.15,  sp.Matrix([0,0,1]), 
              inertia_matrix=Icube_eval(0.15, 0.027, 0.0385, 0.077))

### Motors

In [60]:
# Motors 2*50g at z=0.025 (m, l, r1, r2)
b_mot = Body(0.1,  sp.Matrix([0,0,0.025]), 
             inertia_matrix=Icyl_eval(.1, 0.04, 0.0, 0.05)  )

In [61]:
# flight controller and escs 100g at z=0.600
b_fc = Body(0.05,  sp.Matrix([0,0,0.6]))

In [62]:
# rocket body
cylinder_m = 0.1
cylinder_length = 0.8
cylinder_r2 = 0.025
cylinder_r1 = 0.024

# the rotational inertia matrix around the cg of the cylinder
cyl_i = Icyl_eval(cylinder_length, cylinder_r1, cylinder_r2, cylinder_m)

In [63]:
# vector to the cylinder center of mass in world coord
# assumes bottom of cylinder is 0.2m above z0
cyl_above_z0 = 0.2
r = sp.Matrix([0,0,cylinder_length/2+cyl_above_z0])
# Create the body
b_cyl = Body(cylinder_m, r=r, inertia_matrix=cyl_i)

In [64]:
# create the list of all the bodies
body_list = [b_batt, b_mot, b_fc, b_cyl]

In [65]:
cg = calcCG(body_list)
cg

⎡   0   ⎤
⎢       ⎥
⎢   0   ⎥
⎢       ⎥
⎣0.60625⎦

The CG is located 606mm from the ground

In [66]:
calcI(body_list, cg=cg)

array([[0.0595339489583333, 0, 0],
       [0, 0.0595245333333333, 0],
       [0, 0, 0.00440264062500000]], dtype=object)

## Get more complicated
Add a 3D printed ring of four servos (28g each) on a structural circle of diameter 200m 

In [67]:
# ring
cylinder_m = 0.150
cylinder_length = 0.01
cylinder_r2 = 0.110
cylinder_r1 = 0.1

# the rotational inertia matrix around the cg of the cylinder
ring_i = Icyl_eval(cylinder_m, cylinder_length, cylinder_r1, cylinder_r2 )

# vector to the cylinder center of mass in world coord
# assumes bottom of cylinder is at z0
r = sp.Matrix([0,0,cylinder_length/2])
# Create the body
b_ring = Body(cylinder_m, r=r, inertia_matrix=ring_i)

In [68]:
# 4 servos
# Motors 2*50g at z=0.025
b_serv1 = Body(0.025,  sp.Matrix([0.1,0,0]))
b_serv2 = Body(0.025,  sp.Matrix([-0.1,0,0]))
b_serv3 = Body(0.025,  sp.Matrix([0,0.1,0]))
b_serv4 = Body(0.025,  sp.Matrix([0,-0.1,0]))

In [69]:
# create the list of all the bodies
body_list =  [b_batt, b_mot, b_fc, b_cyl, b_ring, b_serv1, b_serv2, b_serv3,  b_serv4]

In [70]:
cg = calcCG(body_list)
cg

⎡        0        ⎤
⎢                 ⎥
⎢        0        ⎥
⎢                 ⎥
⎣0.374230769230769⎦

In [80]:
rocket_I = calcI(body_list, cg=cg)
rocket_I

array([[0.116855439342949, 0, 0],
       [0, 0.116846023717949, 0],
       [0, 0, 0.00706014062500000]], dtype=object)

In [75]:
import pickle

In [82]:
pickle.dump(rocket_I, open('rocket_I.p', 'wb'))

In [77]:
test = pickle.load(open('test.p', 'rb'))

In [78]:
test

array([[0.0833333333333333*h*(3*m**2 + 3*r**2), 0, 0],
       [0, 0.0833333333333333*h*(3*m**2 + 3*r**2), 0],
       [0, 0, 0.5*h*(m**2 + r**2)]], dtype=object)